In [1]:
import xarray as xr
import torch
from torch.nn import functional as F
from torch import nn
from torch import optim
import numpy as np
import matplotlib.pyplot as plt
import os
import argparse
from datetime import date
import json
import copy
from scipy.interpolate import RectBivariateSpline
import scipy
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import product

os.system("jupyter nbconvert --to script 'climate_train.ipynb'")
os.system("jupyter nbconvert --to script 'climate_data.ipynb'")
os.system("jupyter nbconvert --to script 'climate_models.ipynb'")

import climate_train
import climate_data
import climate_models

In [2]:
def jobnums(C=[2,3,4,3,2],x=[[0,1],[0,1,2],[0,1,2,3],[1,2],[0]],offset=5000):
    lxi=[len(xx) for xx in x]
    n=np.prod(lxi)
    y=[]
    for i in range(n):
        ii=i
        tt=0
        TT=1
        for j in range(len(x)):
            tt+=x[j][ii%lxi[j]]*TT
            ii=ii//lxi[j]
            TT*=C[j]
        y.append(tt)
    y=[tt+offset for tt in y] 
    return y

def configure_models(modelnums):
    for i in modelnums:
        string_input="--b 1 -e 10 --nworkers 10 \
            --subtime 0.005 --disp 1 --relog 1 --rerun 1 \
            --lr 0.01 --model_id " +str(i)+ " --model_bank_id G"
        args=climate_train.options(string_input=string_input.split())
        _=climate_models.model_bank(args,configure=True,verbose=False)

        net,criterion,(data_init,partition),logs,(PATH0,PATH1,LOG,root)=climate_train.load_from_save(args)
        print(net.nparam,net.receptive_field)
        (training_set,training_generator),\
                    (val_set,val_generator),\
                        (test_set,test_generator),\
                            (glbl_set,glbl_gen)=climate_train.load_data(data_init,partition,args)
        #landmasks=climate_data.get_land_masks(val_generator)
        training_set.save_scales()
        training_set.save_masks()
        landmasks=training_set.get_masks()
        X,dom_num,Y=training_set[0]
        print('\t\t'+str(landmasks[dom_num].shape)+' '+str(Y.shape))
        print(str(i),flush=True)

def check_model(modelnums):
    final_lrs=[]
    num_iter=[]
    eval_vals=[]
    
    for i in range(len(modelnums)):
        if modelnums[i]//1000!=6:
            LOG='/scratch/cg3306/climate/runs/G-'+str(modelnums[i])+'/log.json'

            try:
                with open(LOG, 'r') as outfile:
                    logs=json.load(outfile)
                if np.isnan(logs['test-loss'][-1]):
                    flr=np.nan
                    niter=np.nan
                else:
                    flr=logs['lr'][-1]
                    niter=len(logs['lr'])
            except:
                flr=1
                niter=0
            final_lrs.append(flr)
            num_iter.append(niter)
            DIR='/scratch/cg3306/climate/runs/G-'+str(modelnums[i])+'/'
            evalflag='MSE.npy' in os.listdir(DIR)
            evalflag=evalflag and ('MSE-co2.npy' in os.listdir(DIR))
            eval_vals.append(evalflag)
        else:
            final_lrs.append(0)
            DIR='/scratch/cg3306/climate/runs/G-'+str(modelnums[i])+'/'
            num=0
            if 'X2-train.npy' in os.listdir(DIR):
                num+=1
            if 'X2-val.npy' in os.listdir(DIR):
                num+=1
            if 'X2-test.npy' in os.listdir(DIR):
                num+=1
            num_iter.append(num)
            evalflag=('MSE.npy' in os.listdir(DIR))+  ('MSE-co2.npy' in os.listdir(DIR))*2
            eval_vals.append(evalflag)
    return final_lrs,num_iter,eval_vals
def select_models(C,x,conditional,offset):
    modelnums=jobnums(C=C,x=x,offset=offset)
    modelnums=np.array(modelnums)
    flr,niter,evals=check_model(modelnums)
    y=np.array([conditional(flr[i],niter[i],evals[i]) for i in range(len(flr))])
    I=np.where(y)[0]
    return modelnums[I]
def report_progress(C,x,offset=5000):
    report_short=lambda str_,I:  print(str_+':\n'+str(I.tolist()).replace(' ',''))
    if offset==6000:
        Instrt=select_models(C,x,lambda x,y,z: y==0,offset)
        Inonfinished=select_models(C,x,lambda x,y,z: y<3,offset)
        Ifinished=select_models(C,x,lambda x,y,z: y==3,offset)
        Istrt=select_models(C,x,lambda x,y,z: y>0,offset)
        
        Ieval0=select_models(C,x,lambda x,y,z: not z,offset)
        Ieval1=select_models(C,x,lambda x,y,z: z,offset)
        report_short('Hasnt started',Instrt)
        report_short('Files exist',Istrt)
        report_short('No test file',Inonfinished)
        report_short('Test file',Ifinished)
        report_short('Done eval',Ieval1)
        report_short('No eval',Ieval0)
        return
    Instp=select_models(C,x,lambda x,y,z: x>1e-7,offset)
    Ilate_eval=select_models(C,x,lambda x,y,z: x>1e-7 and x<1e-3 and not z,offset)

    Instrt=select_models(C,x,lambda x,y,z: y==0,offset)
    Ieval0=select_models(C,x,lambda x,y,z: (x<=1e-7) and not z,offset)
    Ieval1=select_models(C,x,lambda x,y,z: (x<=1e-7) and z,offset)
    Ieval0_expand=np.concatenate([Ieval0,Ieval0+500],axis=0)
    Ilate_eval=np.concatenate([Ilate_eval,Ilate_eval+500],axis=0)
    Instp_expand=np.concatenate([Instp,Instp+500],axis=0)
    Inan=select_models(C,x,lambda x,y,z: np.isnan(y),offset)
    
    report_short('Hasnt started',Instrt)
    report_short('Hasnt finished',Instp)
    report_short('Nan',Inan)
    report_short('Needs eval',Ieval0_expand)
    report_short('Done eval',Ieval1)
    report_short('Late eval job',Ilate_eval)

In [3]:
'''configure_models(jobs)'''

'configure_models(jobs)'

In [6]:
Test=[6003,6005,6007,6027,6028,6029,6030,6031]

In [7]:
Test+=[tt+500 for tt in Test]

In [8]:
str(Test).replace(' ','')

'[6003,6005,6007,6027,6028,6029,6030,6031,6503,6505,6507,6527,6528,6529,6530,6531]'

In [9]:
C=[2,1,4,3,2]
x=[[0,1],[0],[1,2,3],[0],[0,1]]
report_progress(C,x,offset=6000)

Hasnt started:
[]
Files exist:
[6002,6003,6004,6005,6006,6007,6026,6027,6028,6029,6030,6031]
No test file:
[]
Test file:
[6002,6003,6004,6005,6006,6007,6026,6027,6028,6029,6030,6031]
Done eval:
[6002,6004,6006,6007,6026,6028,6031]
No eval:
[6003,6005,6027,6029,6030]


In [6]:
jobs=jobnums(C=C,x=x,offset=6000)

In [7]:
6003,6005,6007,6027,6028,6029,6030,6031

[6008,
 6009,
 6010,
 6011,
 6012,
 6013,
 6014,
 6015,
 6016,
 6017,
 6018,
 6019,
 6020,
 6021,
 6022,
 6023,
 6032,
 6033,
 6034,
 6035,
 6036,
 6037,
 6038,
 6039,
 6040,
 6041,
 6042,
 6043,
 6044,
 6045,
 6046,
 6047]

In [6]:
str(jobs).replace(' ','')

'[6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6032,6033,6034,6035,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047]'

In [8]:
C=[2,1,4,3,2]
x=[[0,1],[0],[0,1,2,3],[0],[0,1]]
report_progress(C,x,offset=6000)

Hasnt started:
[]
Files exist:
[6000,6001,6002,6003,6004,6005,6006,6007,6024,6025,6026,6027,6028,6029,6030,6031]
No test file:
[6000,6001,6002,6003,6004,6005,6006,6007,6024,6025,6026,6027,6028,6029,6030,6031]
Done eval:
[]
No eval:
[6000,6001,6002,6003,6004,6005,6006,6007,6024,6025,6026,6027,6028,6029,6030,6031]


In [ ]:
C=[2,3,4,3,2]
x=[[0,1],[0,2],[0,1,2,3],[2],[1]]
report_progress(C,x)

In [11]:
y1=jobnums(C=[2,3,4,3,2],x=[[0,1],[0,2],[0,1,2,3],[2],[1]])
print(str(y1).replace(' ',''))

modelnums=y1
modelnums=np.array(modelnums)
flr,niter,evals=check_model(modelnums)
Inan=np.where(np.isnan(np.array(niter)))[0]
Instrt=np.where(np.array(niter)==0)[0]
Ieval=np.where(np.array(evals)==False)[0]
Ilr=np.where(np.array(flr)<1)[0]
Infinish=np.where(np.array(flr)>1e-7)[0]
I=np.where((np.array(flr)>1e-7))[0]
Iruneval=np.where(np.array(flr)<=1e-7)[0]
for i in I:
    print(str(modelnums[i])+'\t\t'+str(flr[i])+'\t\t'+str(niter[i]))
#G-5 Experiment 1
I1=modelnums[I]
I1nstrt=modelnums[Instrt]
I1lr=modelnums[Ilr]
I1nan=modelnums[Inan]
I1eval=modelnums[Ieval]
I1nfinish=modelnums[Infinish]
I1runeval=modelnums[Iruneval]
I1runeval=np.concatenate([I1runeval,I1runeval+500],axis=0)

[5120,5121,5124,5125,5126,5127,5130,5131,5132,5133,5136,5137,5138,5139,5142,5143]
5120		3.90625e-05		114
5124		0.0025		54
5125		2.44140625e-06		266
5126		6.103515625e-07		262
5130		0.000625		88


In [12]:
print(str(I1.tolist()).replace(' ',''))
print(str(I1nstrt.tolist()).replace(' ',''))
print(str(I1lr.tolist()).replace(' ',''))
print(str(I1eval.tolist()).replace(' ',''))
print(str(I1nan.tolist()).replace(' ',''))
print(str(I1nfinish.tolist()).replace(' ',''))
print(str(I1runeval.tolist()).replace(' ',''))

[5120,5124,5125,5126,5130]
[]
[5120,5121,5124,5125,5126,5127,5130,5131,5132,5133,5136,5137,5138,5139,5142,5143]
[5120,5121,5124,5125,5126,5127,5130,5131,5132,5133,5136,5137,5138,5139,5142,5143]
[]
[5120,5124,5125,5126,5130]
[5121,5127,5131,5132,5133,5136,5137,5138,5139,5142,5143,5621,5627,5631,5632,5633,5636,5637,5638,5639,5642,5643]


In [3]:
y1=jobnums(C=[2,3,4,3,2],x=[[0,1],[0,1],[0,1,2,3],[0],[1]])
print(str(y1).replace(' ',''))

modelnums=y1
modelnums=np.array(modelnums)
flr,niter,evals=check_model(modelnums)
Inan=np.where(np.isnan(np.array(niter)))[0]
Ieval=np.where(np.array(evals)==False)[0]
I=np.where(np.array(flr)>1e-7)[0]
for i in I:
    print(str(modelnums[i])+'\t\t'+str(flr[i])+'\t\t'+str(niter[i]))
#G-5 Experiment 1
I1=modelnums[I]
I1nan=modelnums[Inan]
I1eval=modelnums[Ieval]

[5072,5073,5074,5075,5078,5079,5080,5081,5084,5085,5086,5087,5090,5091,5092,5093]
5072		7.8125e-05		201
5074		0.000625		217


In [4]:
print(str(I1.tolist()).replace(' ',''))
print(str(I1eval.tolist()).replace(' ',''))
print(str(I1nan.tolist()).replace(' ',''))

[5072,5074]
[]
[]


In [5]:
str(y1+[y+500 for y in y1]).replace(' ','')

'[5072,5073,5074,5075,5078,5079,5080,5081,5084,5085,5086,5087,5090,5091,5092,5093,5572,5573,5574,5575,5578,5579,5580,5581,5584,5585,5586,5587,5590,5591,5592,5593]'

In [16]:
y1=jobnums(C=[2,3,4,3,2],x=[[0,1],[0,2],[0,1,2,3],[2],[1]])
print(str(y1).replace(' ',''))

modelnums=y1
modelnums=np.array(modelnums)
flr,niter,evals=check_model(modelnums)
Inan=np.where(np.isnan(np.array(niter)))[0]
Ieval=np.where(np.array(evals)==False)[0]
I=np.where(np.array(flr)>1e-7)[0]
for i in I:
    print(str(modelnums[i])+'\t\t'+str(flr[i])+'\t\t'+str(niter[i]))
#G-5 Experiment 1
I1=modelnums[I]
I1nan=modelnums[Inan]
I1eval=modelnums[Ieval]

[5120,5121,5124,5125,5126,5127,5130,5131,5132,5133,5136,5137,5138,5139,5142,5143]
5120		1		0
5121		1		0
5124		1		0
5125		1		0
5126		1		0
5127		1		0
5130		1		0
5131		1		0
5132		1		0
5133		1		0
5136		1		0
5137		1		0
5138		1		0
5139		1		0
5142		1		0
5143		1		0


In [7]:
str(y1+[y+500 for y in y1]).replace(' ','')

'[5096,5097,5098,5099,5100,5101,5102,5103,5104,5105,5106,5107,5108,5109,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119,5596,5597,5598,5599,5600,5601,5602,5603,5604,5605,5606,5607,5608,5609,5610,5611,5612,5613,5614,5615,5616,5617,5618,5619]'

In [8]:
print(str(I1.tolist()).replace(' ',''))
print(str((I1eval).tolist()).replace(' ',''))
print(str(I1nan.tolist()).replace(' ',''))

[5096,5098,5100,5106]
[]
[]


In [9]:
y=jobnums(C=[2,3,4,3,2],x=[[0,1],[0,1,2],[0,1,2,3],[1],[1]])
str(y).replace(' ','')

'[5096,5097,5098,5099,5100,5101,5102,5103,5104,5105,5106,5107,5108,5109,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119]'

In [11]:
#G-5 Experiment 2
modelnums=[5024,5025,5026,5027,5028,5029,5030,5031,5032,5033,5034,5035,5036,5037,5038,5039,\
                5040,5041,5042,5043,5044,5045,5046,5047,5048,5049,5052,5053,5054,5055,5058,5059,\
                5060,5061,5064,5065,5066,5067,5070,5071]
modelnums=np.array(modelnums)
flr,niter=check_model(modelnums)
Inan=np.where(np.isnan(np.array(niter)))[0]
I=np.where(np.array(flr)>1e-7)[0]
for i in I:
    print(str(modelnums[i])+'\t\t'+str(flr[i])+'\t\t'+str(niter[i]))
#G-5 Experiment 1
I1=modelnums[I]
I1nan=modelnums[Inan]
modelnums=[5000,5001,5002,5003,5006,5007,5008,5009,5012,5013,5014,5015,5018,5019,5020,5021]
modelnums=np.array(modelnums)
flr,niter=check_model(modelnums)

Inan=np.where(np.isnan(np.array(niter)))[0]
I=np.where(np.array(flr)>1e-7)[0]
I1=np.concatenate([modelnums[I],I1],axis=0)
I1nan=np.concatenate([modelnums[Inan],I1nan],axis=0)
for i in I:
    print(str(modelnums[i])+'\t\t'+str(flr[i])+'\t\t'+str(niter[i]))

ValueError: too many values to unpack (expected 2)

In [12]:
str(I1.tolist()).replace(' ','')

'[5096,5098,5100,5106]'

In [5]:
str(I1nan.tolist()).replace(' ','')

'[5026]'

In [12]:
str(np.array(modelnums)[np.where((np.array(niter)==0) + (np.array(niter)==np.nan))[0]].tolist()).replace(' ','')

'[]'

In [17]:
outputs=['Su','Sv','ST']

In [18]:
print(outputs[0]=='Su')

True
